### Imports

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

#### Data imports

In [2]:
emissions={
'Eolien en mer': 0.016740757290645353,
 'Eolien terrestre': 0.016345944160376588,
 'Fioul': 1.0413904849794613,
 'Charbon': 1.0970894476163344,
 'Autre renouvelables': 0.2346071813871875,
 'Centrales gaz': 0.626227083593243,
 'Nucleaire': 0.006463640014307128,
 'Hydraulique': 0.004332878603214016,
 'Solaire': 0.09029513144067704}

## Insert dict for 2030-2040

## Insert dict for 2040-2050


In [3]:
dataset=xr.open_dataset('elec_mixes2020_2050.nc')
display(dataset)

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 ...

In [ ]:
Date_vals = dataset.coords['Date'].values
year_vals = dataset.coords['year'].values
Hour_vals = dataset.coords['Hour'].values
technology_vals = dataset.coords['technology'].values
#print(Date_vals,year_vals,Hour_vals,technology_vals)

In [4]:
# Extract the key-value pairs to list in an orderly manner : 
list1= []
list2 = []

for key, val in emissions.items() : 
    if key !='Transmission':
        print(key,val)
        list1.append(key)
        list2.append(val)
    
# Make an xarray of the emission factors for each technology : 
array = xr.DataArray(
    list2,
    coords = [list1],
    dims = ['technology']
)
CO2_per_mix = dataset.copy()
CO2_per_mix = CO2_per_mix*array
display(CO2_per_mix)

Eolien en mer 0.016740757290645353
Eolien terrestre 0.016345944160376588
Fioul 1.0413904849794613
Charbon 1.0970894476163344
Autre renouvelables 0.2346071813871875
Centrales gaz 0.626227083593243
Nucleaire 0.006463640014307128
Hydraulique 0.004332878603214016
Solaire 0.09029513144067704


<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 0.0...

### Export to netcdf

In [8]:
CO2_per_mix.to_netcdf(path = 'CO2_per_mix.nc')

#### Validations

In [13]:
test_year = 2050
test_tech = 'Autre renouvelables'
test_date = 354
test_hour = 3

Valid_CO2=CO2_per_mix.sel(
    technology=test_tech,
    year= test_year,
    Date=test_date,
    Hour=test_hour,
).to_array().values[0]

Valid_share = dataset.sel(
    technology=test_tech,
    year= test_year,
    Date=test_date,
    Hour=test_hour,
).to_array().values[0]

print(test_tech,Valid_CO2,Valid_share*emissions[test_tech])

Autre renouvelables 0.005192473735170873 0.005192473735170873
